In [1]:
"""full 1 channel tiff writer without dataframe log"""



def create_marked_file_final(xml_file, input_file, output_file):
    import numpy as np
    import tifffile
    import xml.etree.ElementTree as ET

    original_image = tifffile.imread(input_file)
    layers, channels, height, width = original_image.shape
    print(original_image.shape)
    binary_image = np.zeros((layers, 1, height, width), dtype=np.uint8)
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for marker_type in root.findall('Marker_Data/Marker_Type'):
        type_id = int(marker_type.find('Type').text)

        for marker in marker_type.findall('Marker'):

            x = int(marker.find('MarkerX').text) -1
            y = int(marker.find('MarkerY').text) -1
            z_raw = ((int(marker.find('MarkerZ').text) + 2 ) // 3) -1

            binary_image[z_raw, 0, y, x] = 255

    tifffile.imwrite(output_file, binary_image)
    return binary_image


create_marked_file_final('FF15A/CellCounter_FF151L2L.xml', 'FF15/FF151L2L.lsm', 'FF15/FF151L2L_marked7.tif')



(56, 3, 512, 512)


array([[[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]],


       [[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]],


       [[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]],


       ...,


       [[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]],


       [[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 

In [2]:
"""Full dataframe logger from xml file"""



def pull_marker_data(xml_file):
    import xml.etree.ElementTree as et
    import pandas as pd
    file = et.parse(xml_file)
    root = file.getroot()
    marker_data = []

    MDElement = root.find('Marker_Data')
    for Marker_Type in MDElement.findall('Marker_Type'):
        type_id = int(Marker_Type.find('Type').text)

        for Marker in Marker_Type.findall('Marker'):
            xloc = int(Marker.find('MarkerX').text) if Marker.find('MarkerX') is not None else None
            yloc = int(Marker.find('MarkerY').text) if Marker.find('MarkerY') is not None else None
            zloc = int(Marker.find('MarkerZ').text) if Marker.find('MarkerZ') is not None else None
            zlayer = (2 + zloc) // 3

            marker_data.append({
                'z': zlayer,
                'type_id': type_id,
                'y': yloc,
                'x': xloc,
            })

    dataframe = pd.DataFrame(marker_data, columns=['z', 'type_id', 'y', 'x'])
    return dataframe

result = pull_marker_data('FF15A/CellCounter_FF151L2L.xml')
result.head(96)

,z,type_id,y,x
0,11,1,206,115
1,11,1,213,114
2,11,1,229,99
3,11,1,226,82
4,11,1,223,80
...,...,...,...,...
91,32,10,356,268
92,32,10,354,271
93,32,10,363,259
94,32,10,385,258


In [11]:
result = pull_marker_data('FF15/CellCounter_FF151L2L_marked92.xml')
result.head(96)

,z,type_id,y,x
0,13,1,80,249
1,13,1,85,250
2,13,1,93,246
3,13,1,107,232
4,13,1,108,227
5,13,1,74,245


In [27]:
"""full 1 channel tiff writer from dataframe log"""


def pull_synapse_and_mark_file(xml_file, input_file, output_file):
    import numpy as np
    import tifffile

    df = pull_marker_data(xml_file)

    original_image = tifffile.imread(input_file)
    layers, channels, height, width = original_image.shape
    binary_image = np.zeros((layers, 1, height, width), dtype=np.uint8)

    x = df['x']
    y = df['y']
    z = df['z']

    for i in range(len(df)):
        binary_image[z[i] - 1,:, y[i] - 1, x[i] - 1] = 255

    tifffile.imwrite(output_file, binary_image)
    return binary_image

# Example usage
pull_synapse_and_mark_file('FF15A/CellCounter_FF151L2L.xml', 'FF15/FF151L2L.lsm', 'FF15/FF151L2L_marked9.tif')

array([[[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]],


       [[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]],


       [[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]],


       ...,


       [[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]],


       [[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 

In [2]:
"""full 3 channel tiff writer without dataframe log"""

def create_colored_file_final(xml_file, input_file, output_file):
    import numpy as np
    import tifffile
    import xml.etree.ElementTree as ET

    original_image = tifffile.imread(input_file)
    layers, channels, height, width = original_image.shape
    print(original_image.shape)
    rgb_image = np.zeros((layers, 3, height, width), dtype=np.uint8)
    tree = ET.parse(xml_file)
    root = tree.getroot()
    typecount = 0
    for marker_type in root.findall('Marker_Data/Marker_Type'):
        typecount += 1
        continue
    type_array = []
    start_color = np.array([255, 0, 0])
    end_color = np.array([0, 0, 255])
    for i in range(typecount):
        color = (start_color + (end_color - start_color) * i / (typecount - 1)).astype(int)
        type_array.append(tuple(color))
    for marker_type in root.findall('Marker_Data/Marker_Type'):
        type_id = int(marker_type.find('Type').text)
        if type_id <= typecount // 2:
            color = (
                np.array([255, 0, 0]) + (np.array([0, 255, 0]) - np.array([255, 0, 0])) * type_id / (typecount - 1)
            ).astype(int)
        else:
            color = (
                np.array([0, 255, 0]) + (np.array([0, 0, 255]) - np.array([0, 255, 0])) * (type_id - (typecount // 2)) / (typecount - 1)
            ).astype(int)

        for marker in marker_type.findall('Marker'):

            x = int(marker.find('MarkerX').text) - 1
            y = int(marker.find('MarkerY').text) - 1
            z = ((int(marker.find('MarkerZ').text) + 2) // 3) - 1

            rgb_image[z, :, y, x] = color  # Apply color to all three channels for the specific pixel

    tifffile.imwrite(output_file, rgb_image)
    return rgb_image


create_colored_file_final('FF15A/CellCounter_FF151L2L.xml', 'FF15/FF151L2L.lsm', 'FF15/FF151L2L_marked72.tif')

(56, 3, 512, 512)


array([[[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]],


       [[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
  

In [3]:
""" create the most distinct colors from the dataframe"""



def find_distinct_colors(xml_file):
    import numpy as np
    import colorsys

    df = pull_marker_data(xml_file)
    num_colors = max(df['type_id'])

    colors = []
    for i in range(num_colors):
        r, g, b = colorsys.hsv_to_rgb(i / num_colors, 1.0, 1.0)
        rgb = np.array([int(r * 255), int(g * 255), int(b * 255)])
        colors.append(rgb)

    return colors


print(find_distinct_colors('FF15A/CellCounter_FF151L2L.xml'))




[array([255,   0,   0]), array([255, 153,   0]), array([203, 255,   0]), array([ 51, 255,   0]), array([  0, 255, 102]), array([  0, 255, 255]), array([  0, 102, 255]), array([ 50,   0, 255]), array([204,   0, 255]), array([255,   0, 152])]


In [4]:
"""full 3 channel hue color tiff writer from dataframe log"""



def pull_synapse_and_mark_file(xml_file, input_file, output_file):
    import numpy as np
    import tifffile
    df = pull_marker_data(xml_file)
    color_array = find_distinct_colors(xml_file)
    original_image = tifffile.imread(input_file)
    layers, channels, height, width = original_image.shape
    rgb_image = np.zeros((layers, 3, height, width), dtype=np.uint8)
    x = df['x']
    y = df['y']
    z = df['z']
    type_id = df['type_id']
    for i in range(len(df)):
        rgb_image[z[i] -1, :, y[i] -1, x[i] -1] = color_array[type_id[i] -1]
    tifffile.imwrite(output_file, rgb_image)
    return rgb_image

pull_synapse_and_mark_file('FF15A/CellCounter_FF151L2L.xml', 'FF15/FF151L2L.lsm', 'FF15/FF151L2L_marked92.tif')
result = pull_marker_data('FF15A/CellCounter_FF151L2L.xml')
result.head(96)

,z,type_id,y,x
0,11,1,206,115
1,11,1,213,114
2,11,1,229,99
3,11,1,226,82
4,11,1,223,80
...,...,...,...,...
91,32,10,356,268
92,32,10,354,271
93,32,10,363,259
94,32,10,385,258
